# 书生"万卷"韩语专业领域大模型 LoRA 微调

## 环境安装（包下载） 
使用https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple 清华园镜像  


In [1]:
!pip install modelscope langdetect peft jieba torch sentence_transformers scikit-learn rouge_chinese nltk -i https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple

Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple


## 模型与数据集
### 使用modelscope下载两个用到的模型（hf有网络问题）

In [2]:
!modelscope download --model sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2


 _   .-')                _ .-') _     ('-.             .-')                              _ (`-.    ('-.
( '.( OO )_             ( (  OO) )  _(  OO)           ( OO ).                           ( (OO  ) _(  OO)
 ,--.   ,--.).-'),-----. \     .'_ (,------.,--.     (_)---\_)   .-----.  .-'),-----.  _.`     \(,------.
 |   `.'   |( OO'  .-.  ',`'--..._) |  .---'|  |.-') /    _ |   '  .--./ ( OO'  .-.  '(__...--'' |  .---'
 |         |/   |  | |  ||  |  \  ' |  |    |  | OO )\  :` `.   |  |('-. /   |  | |  | |  /  | | |  |
 |  |'.'|  |\_) |  |\|  ||  |   ' |(|  '--. |  |`-' | '..`''.) /_) |OO  )\_) |  |\|  | |  |_.' |(|  '--.
 |  |   |  |  \ |  | |  ||  |   / : |  .--'(|  '---.'.-._)   \ ||  |`-'|   \ |  | |  | |  .___.' |  .--'
 |  |   |  |   `'  '-'  '|  '--'  / |  `---.|      | \       /(_'  '--'\    `'  '-'  ' |  |      |  `---.
 `--'   `--'     `-----' `-------'  `------'`------'  `-----'    `-----'      `-----'  `--'      `------'


Successfully Downloaded from model sentence-transfor

In [3]:
!modelscope download --model Qwen/Qwen2.5-1.5B-Instruct


 _   .-')                _ .-') _     ('-.             .-')                              _ (`-.    ('-.
( '.( OO )_             ( (  OO) )  _(  OO)           ( OO ).                           ( (OO  ) _(  OO)
 ,--.   ,--.).-'),-----. \     .'_ (,------.,--.     (_)---\_)   .-----.  .-'),-----.  _.`     \(,------.
 |   `.'   |( OO'  .-.  ',`'--..._) |  .---'|  |.-') /    _ |   '  .--./ ( OO'  .-.  '(__...--'' |  .---'
 |         |/   |  | |  ||  |  \  ' |  |    |  | OO )\  :` `.   |  |('-. /   |  | |  | |  /  | | |  |
 |  |'.'|  |\_) |  |\|  ||  |   ' |(|  '--. |  |`-' | '..`''.) /_) |OO  )\_) |  |\|  | |  |_.' |(|  '--.
 |  |   |  |  \ |  | |  ||  |   / : |  .--'(|  '---.'.-._)   \ ||  |`-'|   \ |  | |  | |  .___.' |  .--'
 |  |   |  |   `'  '-'  '|  '--'  / |  `---.|      | \       /(_'  '--'\    `'  '-'  ' |  |      |  `---.
 `--'   `--'     `-----' `-------'  `------'`------'  `-----'    `-----'      `-----'  `--'      `------'


Successfully Downloaded from model Qwen/Qwen2.5-1.5B

### 书生“万卷”数据集下载

https://opendatalab.com/OpenDataLab/WanJuan-Korean/tree/main/raw/professional_field/general  
part-677f75d865d8-000972.jsonl.gz  

## 环境导入（包导入）

In [4]:
import torch  # PyTorch深度学习框架
from datasets import load_dataset, Dataset  # Hugging Face的数据集加载工具
from transformers import (  # Hugging Face的转换器库
    AutoModelForCausalLM,  # 自回归语言模型（用于生成文本）
    AutoTokenizer,  # 自动分词器
    AutoModelForMaskedLM,  # 掩码语言模型
    get_linear_schedule_with_warmup,  # 学习率预热调度器
    DataCollatorForLanguageModeling  # 用于MLM的数据整理器
)
from peft import LoraConfig, get_peft_model, TaskType  # 参数高效微调工具
from torch.utils.data import DataLoader, random_split  # 数据加载相关工具
from torch.optim import AdamW  # Adam优化器的变体
from tqdm import tqdm  # 进度条工具
import os
import random
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import logging
from datetime import datetime
import json
import random
import re
import os
from pathlib import Path
from tqdm import tqdm
from collections import defaultdict, Counter
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
import math
import jieba

from typing import List, Dict, Set
import langdetect
from langdetect.lang_detect_exception import LangDetectException

/root/.conda/envs/qwenllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  ## 模型评估与优化  （确定评估指标）  
  - 困惑度(Perplexity)  
    - 评估模型在验证集上的表现  
    - 监控模型是否过拟合  
  - 领域适应性和质量评估  
    - 术语覆盖率：# term_coverage是术语覆盖率，计算response中包含的领域术语数量占总术语数量的比例  
    - 术语密度：# term_density是术语密度，计算response中包含的领域术语数量占总token数量的比例  
    - 响应质量：# response_quality是回复质量，计算response与prompt的相似度
    - bleu  
    - rouge1、rouge2、rougel

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_chinese import Rouge
import jieba
import numpy as np
from tqdm import tqdm
import torch

sentence_path = "/root/.cache/modelscope/hub/models/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

class DomainEvaluator:
    def __init__(self, tokenizer, device):
        self.tokenizer = tokenizer
        self.device = device
        # 加载sentence transformer用于计算文本相似度
        self.sentence_model = SentenceTransformer(sentence_path)
        self.sentence_model.to(device)
        
        # 加载领域术语
        with open(ko_domain_path, "r", encoding="utf-8") as f:
            self.domain_terms = [line.strip() for line in f if line.strip()]
        

    def calculate_domain_perplexity(self, model, eval_dataloader, sample_ratio):
        """计算领域数据的困惑度，使用随机采样
        
        Args:
            model: 模型
            eval_dataloader: 评估数据加载器
            sample_ratio: 评估样本比例，默认为0.3（30%）
        """
        model.eval()
        total_loss = 0
        total_tokens = 0
        
        # 获取数据集大小
        dataset_size = len(eval_dataloader.dataset)
        
        # 计算需要采样的样本数量
        sample_size = int(dataset_size * sample_ratio)
        
        # 创建随机采样器
        indices = torch.randperm(dataset_size)[:sample_size]
        sampler = torch.utils.data.SubsetRandomSampler(indices)
    
        # 创建新的数据加载器
        eval_dataloader = torch.utils.data.DataLoader(
            eval_dataloader.dataset,
            batch_size=eval_dataloader.batch_size,
            sampler=sampler,
            num_workers=eval_dataloader.num_workers,
            pin_memory=eval_dataloader.pin_memory
        )
        
        with torch.no_grad():
            for batch in eval_dataloader:
                batch = {k: v.to(self.device) for k, v in batch.items()}
                outputs = model(**batch)
                total_loss += outputs.loss.item() * batch["input_ids"].size(0)
                total_tokens += batch["input_ids"].ne(self.tokenizer.pad_token_id).sum().item()
    
        return torch.exp(torch.tensor(total_loss / total_tokens))

    def split_prompt_and_reference(self, batch):
        """
        batch: 一个由 Trainer / DataLoader 返回的 batch，
            至少包含 input_ids 和 labels，二者形状都为 [B, L]

        返回:
            prompts     -> List[str]
            references  -> List[str]
        """
        tokenizer = self.tokenizer

        input_ids = batch["input_ids"]        # [B, L]
        labels    = batch["labels"]           # [B, L]

        prompts, references = [], []

        pad_id = tokenizer.pad_token_id
        eos_id = tokenizer.eos_token_id

        for ids_tensor, lab_tensor in zip(input_ids, labels):
            ids = ids_tensor.tolist()         # 当前样本的 token id 列表
            lab = lab_tensor.tolist()

            # ───── 1. 参考答案（reference） ─────
            #   labels 中 = -100 的位置是 ignore，其他位置是答案 token
            answer_ids = [
                tok for tok, lb in zip(ids, lab)
                if lb != -100 and tok != pad_id
            ]
            # 截到第一个 EOS，防止把 padding decode 进去
            if eos_id in answer_ids:
                answer_ids = answer_ids[: answer_ids.index(eos_id)]
            references.append(
                tokenizer.decode(answer_ids, skip_special_tokens=True).strip()
            )

            # ───── 2. Prompt / 问题 ─────
            #   prompt 位于答案之前：找到 labels 中第一处 ≠ -100 的索引
            try:
                ans_start = lab.index(next(filter(lambda x: x != -100, lab)))
            except StopIteration:
                # 万一整条样本全是 -100（极少见），就把全文当 prompt
                ans_start = len(ids)

            prompt_ids = ids[:ans_start]
            if eos_id in prompt_ids:          # 截掉 prompt 内部可能出现的 <eos>
                prompt_ids = prompt_ids[: prompt_ids.index(eos_id)]

            prompts.append(
                tokenizer.decode(prompt_ids, skip_special_tokens=True).strip()
            )

        return prompts, references

    def evaluate_domain_adaptation(self, model, eval_dataloader, sample_ratio=1.0):
        """
        批量评估生成文本的领域适应性和文本质量（BLEU + ROUGE）
        
        Args:
            model: 语言模型
            eval_dataloader: 数据加载器，batch中需含有 'input_ids' 和 'labels' 字段
            sample_ratio: 采样比例（默认全部评估）
        
        Returns:
            dict: 各项指标的平均值
        """
        model.eval()

        metrics = {
            "term_coverage": [],
            "term_density": [],
            "response_quality": [],
            "bleu": [],
            "rouge-1": [],
            "rouge-2": [],
            "rouge-l": []
        }

        rouge = Rouge()
        smooth = SmoothingFunction()

        # 随机采样子集
        dataset_size = len(eval_dataloader.dataset)
        sample_size = int(dataset_size * sample_ratio)
        indices = torch.randperm(dataset_size)[:sample_size]
        sampler = torch.utils.data.SubsetRandomSampler(indices)

        sampled_dataloader = torch.utils.data.DataLoader(
            eval_dataloader.dataset,
            batch_size=eval_dataloader.batch_size,
            sampler=sampler,
            num_workers=eval_dataloader.num_workers,
            pin_memory=eval_dataloader.pin_memory
        )

        with torch.no_grad():
            for batch in tqdm(sampled_dataloader, desc="Evaluating"):
                batch = {k: v.to(self.device) for k, v in batch.items()}
                
                inputs_text, references = self.split_prompt_and_reference(batch)
                # print("iiiii=======================================", inputs_text[0])
                # print("rrrrr=======================================", references[0])
                # enc = self.tokenizer(
                #         inputs_text,                       # List[str]
                #         return_tensors="pt",
                #         padding=True,
                #         truncation=True,
                #         max_length=512         # ↙ 视你的显存而定
                # ).to(model.device)

                # 生成候选回复
                outputs = model.generate(
                    input_ids=batch["input_ids"],
                    attention_mask=batch.get("attention_mask", None),
                    max_length=800,
                    num_return_sequences=1,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.95
                )
                candidates = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)

                for input_text, reference, candidate in zip(inputs_text, references, candidates):
                    # print("input_text:", input_text)
                    # print("reference:", reference)
                    # print("candidate:", candidate)
                    # ==== 领域术语指标 ====
                    term_count = sum(1 for term in self.domain_terms if term.lower() in candidate.lower())
                    term_coverage = term_count / len(self.domain_terms) if self.domain_terms else 0
                    term_density = term_count / len(candidate) if candidate else 0
                    metrics["term_coverage"].append(term_coverage)
                    metrics["term_density"].append(term_density)

                    # ==== BLEU ====
                    ref_tokens = list(jieba.cut(reference))
                    cand_tokens = list(jieba.cut(candidate))
                    try:
                        bleu_score = sentence_bleu([ref_tokens], cand_tokens, smoothing_function=smooth.method1)
                    except Exception as e:
                        print(f"BLEU计算错误: {e}")
                        bleu_score = 0.0
                    metrics["bleu"].append(bleu_score)

                    # ==== ROUGE ====
                    try:
                        rouge_scores = rouge.get_scores(candidate, reference)[0]
                        metrics["rouge-1"].append(rouge_scores['rouge-1']['f'])
                        metrics["rouge-2"].append(rouge_scores['rouge-2']['f'])
                        metrics["rouge-l"].append(rouge_scores['rouge-l']['f'])
                    except Exception as e:
                        print(f"ROUGE计算错误: {e}")
                        metrics["rouge-1"].append(0.0)
                        metrics["rouge-2"].append(0.0)
                        metrics["rouge-l"].append(0.0)

                # ==== 语义相似度 ====
                response_embeddings = self.sentence_model.encode(candidates, convert_to_tensor=True)
                reference_embeddings = self.sentence_model.encode(references, convert_to_tensor=True)
                sims = cosine_similarity(response_embeddings.cpu().numpy(), reference_embeddings.cpu().numpy())
                diagonal_sims = np.diag(sims)
                metrics["response_quality"].extend(diagonal_sims.tolist())

        return {
            "avg_term_coverage": np.mean(metrics["term_coverage"]),
            "avg_term_density": np.mean(metrics["term_density"]),
            "avg_response_quality": np.mean(metrics["response_quality"]),
            "avg_bleu": np.mean(metrics["bleu"]),
            "avg_rouge-1": np.mean(metrics["rouge-1"]),
            "avg_rouge-2": np.mean(metrics["rouge-2"]),
            "avg_rouge-l": np.mean(metrics["rouge-l"]),
        }

## 数据处理  
### 数据处理流程  
- 数据清洗  
- 去除HTML标签和特殊字符  
- 规范化文本格式  
- 过滤过长或过短的文本  
- 数据转换  
- 构建输入输出对  
- 添加任务相关的指令提示

In [6]:
import json
import gzip
from pathlib import Path
import re
from tqdm import tqdm
import random
import os
import pandas as pd

# 输入输出路径
base_dir = "/root/ko_text"  # 原始数据集目录
clean_data_path = "/root/ko_text/ko.json"  # 输出文件路径

def clean_text(text):
    """清理文本内容"""
    if not text:
        return text
    
    # 正则表达式是一种文本匹配模式,下面详细解释每一步:
    
    # 1. 处理连续换行符
    # re.sub()函数用于替换文本,接受3个参数:
    # - 第1个参数 r'\n+' 表示:
    #   \n 代表换行符
    #   + 表示匹配1个或多个连续的换行符
    # - 第2个参数 '\n' 表示用单个换行符替换
    # - 第3个参数是要处理的文本
    # strip()去除文本首尾的空格
    text = re.sub(r'\n+', '\n', text.strip())
    
    # 2. 处理连续空格
    # r'\s+' 表示:
    # \s 代表任意空白字符(空格、制表符等)
    # + 表示匹配1个或多个连续的空白字符
    # 用单个空格替换所有连续的空白字符
    text = re.sub(r'\s+', ' ', text)
    
    # 3. 移除HTML标签，正则表达式 r'<[^>]+>'
    # 1) r'' 表示这是一个原始字符串,不会对反斜杠\进行转义处理
    # 2) < 就是匹配HTML标签的开始符号 <
    # 3) [^>] 是一个字符集:
    #    - [] 表示匹配其中的任意一个字符
    #    - ^ 在[]内表示"非",即取反
    #    - 所以[^>]表示匹配任何不是>的字符
    # 4) + 表示"一个或多个",即重复前面的[^>]一次或多次
    # 5) > 就是匹配HTML标签的结束符号 >
    # 
    # 举例说明:
    # 原文本: "这是<p>一个段落</p>"
    # - <p> 会被匹配,因为它符合模式:<加上任意非>字符(这里是p)再加上>
    # - </p> 也会被匹配,因为它符合模式:<加上任意非>字符(这里是/p)再加上>
    # 
    # re.sub()会把所有匹配到的内容替换为空字符串'',所以最后变成:
    # "这是一个段落"
    text = re.sub(r'<[^>]+>', '', text)
    return text.strip()

def process_item(item):
    """处理单条数据，统一不同数据集的格式"""
    try:
        # 检查数据格式并提取必要字段
        if isinstance(item, dict):
            # 常见格式：包含title和content的字典
            if 'title' in item and 'content' in item:
                title = clean_text(item['title'])
                content = clean_text(item['content'])
            # 其他可能的格式
            elif 'text' in item:
                # 如果只有text字段，尝试从文本中提取标题
                text = clean_text(item['text'])
                lines = text.split('\n', 1)
                if len(lines) > 1:
                    title, content = lines
                else:
                    title = "文章"
                    content = text
            else:
                return None
        else:
            return None
            
        # 检查文本长度
        if len(content) < 50 or len(content) > 800:
            return None
            
        # 构建统一的训练格式
        conversation = {
            "instruction": f"请生成一段带有韩语专业术语的文本。\n\n标题: {title}",
            "input": "",
            "output": content,
            "category": item.get('labels', {}).get('pjwk_cates', "general")
        }
        
        return conversation
    except Exception as e:
        print(f"处理数据时出错: {e}")
        return None

def process_file(input_path, sample_ratio=0.1):
    """处理单个文件并随机抽样"""
    processed_data = []
    
    try:
        # 读取gzip文件
        with gzip.open(input_path, 'rt', encoding='utf-8') as f:
            # 首先读取所有行
            lines = f.readlines()
            
            # 随机抽样
            # 确保sample_size至少为1,避免抽样失败
            sample_size = max(1, int(len(lines) * sample_ratio))
            # 从lines列表中随机抽取sample_size条数据
            sampled_lines = random.sample(lines, sample_size)
            
            # 处理抽样的数据
            for line in tqdm(sampled_lines, desc=f"处理文件 {Path(input_path).name}"):
                try:
                    item = json.loads(line.strip())
                    processed_item = process_item(item)
                    if processed_item:
                        processed_data.append(processed_item)
                except json.JSONDecodeError:
                    continue
                except Exception as e:
                    print(f"处理数据时出错: {e}")
                    continue
    except Exception as e:
        print(f"处理文件 {input_path} 时出错: {e}")
    
    return processed_data

def process_all_datasets(base_dir, output_path, sample_ratio=0.25):
    """处理所有数据集并合并结果"""
    all_processed_data = []
    
    # 递归查找所有.jsonl.gz文件
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith('.jsonl.gz'):
                input_path = os.path.join(root, file)
                print(f"\n处理文件: {input_path}")
                
                # 处理单个文件
                file_data = process_file(input_path, sample_ratio)
                all_processed_data.extend(file_data)
                
                print(f"从 {file} 中提取了 {len(file_data)} 条数据")
    
    # 保存所有处理后的数据
    output_path = Path(output_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(all_processed_data, f, ensure_ascii=False, indent=2)
    
    print(f"\n处理完成！")
    print(f"总共处理了 {len(all_processed_data)} 条数据")
    print(f"数据已保存至: {output_path}")
    
    # 输出数据集统计信息
    categories = {}
    for item in all_processed_data:
        cat = item['category']
        if isinstance(cat, dict):
            cat = str(cat)
        categories[cat] = categories.get(cat, 0) + 1
    
    print("\n数据集类别分布:")
    for cat, count in categories.items():
        print(f"{cat}: {count} 条")

def main():
    # 设置随机种子以确保可重复性
    random.seed(42)
    # 处理所有数据集
    process_all_datasets(base_dir, clean_data_path, sample_ratio=0.1)
    
    # 显示样例数据
    with open(clean_data_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        print("\n处理后的数据样例:")
        print(json.dumps(data[0], ensure_ascii=False, indent=2))

if __name__ == "__main__":
    main() 


处理文件: /root/ko_text/ko_text.jsonl.gz


处理文件 ko_text.jsonl.gz: 100%|██████████| 66033/66033 [00:21<00:00, 3060.01it/s]


从 ko_text.jsonl.gz 中提取了 21248 条数据

处理完成！
总共处理了 21248 条数据
数据已保存至: /root/ko_text/ko.json

数据集类别分布:
{'level1': ['professional_field'], 'level2': ['finance']}: 20358 条
{'level1': ['professional_field'], 'level2': ['medicine']}: 625 条
{'level1': ['professional_field'], 'level2': ['academic']}: 26 条
{'level1': ['professional_field'], 'level2': ['patent']}: 197 条
{'level1': ['professional_field'], 'level2': ['institutions']}: 33 条
{'level1': ['professional_field'], 'level2': ['technology']}: 9 条

处理后的数据样例:
{
  "instruction": "请生成一段带有韩语专业术语的文本。\n\n标题: 공무원연금, 자산관리시스템 대대적 개편",
  "input": "",
  "output": "[아시아경제 박민규 기자] 공무원연금공단이 대대적인 자산관리시스템 개편을 추진한다. 정부의 공무원연금제도 개혁에 발맞춰 조직 혁신에 나선 것이다. 6일 금융투자업계에 따르면 공무원연금은 차세대 금융자산종합관리시스템 구축을 위한 외부 컨설팅을 받을 예정이다. 이달 말까지 입찰 및 제안서 접수 등을 거쳐 컨설팅업체를 선정해 3개월간 컨설팅을 진행하게 된다.공무원연금은 이번 컨설팅을 통해 2005년 도입된 노후화된 자산관리시스템을 개선해 효율적인 금융자산운용 체계 도입과 자산운용업무 전반의 프로세스를 재설계할 방침이다. 이를 바탕으로 공무원연금의 특성에 맞는 자산배분시스템 및 위험관리시스템 등을 구축하는 동시에 금융시장 환경 변화에 능동적으로 대응할 수 있는 운용시스템을 도입하는 방안을 검토한다. 2008~2

### 多线程提取专业术语：如“操作系统”、“高血压” 等等 （我这里使用了传统的特征工程，也可以使用商用的能力较强的大模型判断）

In [9]:
import warnings
warnings.filterwarnings("ignore")

from typing import List
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
from functools import partial
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# 为了语言识别结果的可重复性
DetectorFactory.seed = 0
# 保存术语到 ko.txt
ko_domain_path = "/root/ko_text/ko_domain.txt"

def detect_language(text: str) -> str:
    """自动检测语言，返回语言代码（如 'en', 'zh', 'ko' 等）"""
    try:
        lang = detect(text)
        return lang
    except LangDetectException:
        return "unknown"

# 多进程子任务：在每个进程中初始化 pecab，并处理文本
def extract_terms_worker(text: str, lang: str="ko" ) -> List[str]:
    terms = []
    if lang == 'ko':
        try:
            # 子进程中导入并初始化 pecab
            from pecab import PeCab
            pecab = PeCab()
            pos_tags = pecab.pos(text)
            terms = [word for word, pos in pos_tags if pos.startswith('NN') or pos.startswith('NNP')]
        except Exception as e:
            print(f"[子进程错误] 韩语处理失败: {e}")
    return terms

def parallel_extract(texts: List[str], lang: str="ko", max_workers: int = 16) -> set:
    lang_terms = set()
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        # 使用 partial 函数传 lang 进去，避免 lambda
        func = partial(extract_terms_worker, lang=lang)
        results = list(tqdm(
            executor.map(func, texts),
            total=len(texts),
            desc=f"处理{lang}语言文本"
        ))
    for terms in results:
        lang_terms.update(terms)
    return lang_terms

def extract_terms_parallel_counter(texts: List[str], lang: str="ko", max_workers: int = 16) -> Counter:
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        func = partial(extract_terms_worker, lang=lang)
        results = list(tqdm(
            executor.map(func, texts),
            total=len(texts),
            desc=f"并行提取{lang}语言术语"
        ))

    # 扁平化 + 统计词频
    flat_terms = [term for terms in results for term in terms]
    return Counter(flat_terms)

def extract_domain_terms(domain_texts: List[str], general_texts: List[str] = None, top_n: int = 2000) -> List[str]:
    """使用改进的多语言分词方法提取领域术语"""
    print("开始提取领域术语...")
    
    # 按语言分组处理文本
    lang_texts = defaultdict(list)
    for text in domain_texts:
        lang = detect_language(text)
        if lang == 'ko':
            lang_texts[lang].append(text)
    
    print("\n文本语言分布:")
    for lang, texts in lang_texts.items():
        print(f"- {lang}: {len(texts)} 条")
    
    # 分语言处理并提取术语
    all_terms = []
    for lang, texts in lang_texts.items():
        print(f"\n处理{lang}语言文本...")
        
        # 使用语言专属工具提取术语
        lang_terms = parallel_extract(texts)
        
        # 过滤和排序术语
        lang_terms = list(lang_terms)
        # 计算每个术语在文本中出现的频率
        # 1. 遍历每个文本
        # 2. 对每个文本重新提取术语
        # 3. 使用Counter统计所有术语的频率
        term_freq = extract_terms_parallel_counter(texts)
        
        # 对术语列表进行排序:
        # 1. 首要排序依据是术语出现频率(term_freq[x])
        # 2. 次要排序依据是术语长度(len(x))
        # reverse=True表示按降序排列,即频率高的和长度长的排在前面
        lang_terms.sort(key=lambda x: (term_freq[x], len(x)), reverse=True)
        
        # 根据语言数量平均分配术语数量配额
        # 1. top_n是总的期望术语数量
        # 2. len(lang_texts)是语言种类数
        # 3. 对每种语言,只取配额内的高频长术语
        # //是整除运算符,用于计算每种语言分配的术语数量配额
        # 例如:如果top_n=2000,有4种语言,则每种语言分配2000//4=500个术语
        # 这里的:是切片操作符,表示从列表开头取到指定位置
        # //是整除运算符,例如10//3=3
        # 所以lang_terms[:top_n // len(lang_texts)]表示:
        # 1. 先计算top_n除以语言数量的整除结果n
        # 2. 然后从lang_terms列表中取前n个元素
        selected_terms = lang_terms[:top_n]
        all_terms.extend(selected_terms)
        
        print(f"{lang}语言提取了 {len(selected_terms)} 个术语")
        if selected_terms:
            print(f"{lang}语言术语示例:")
            for term in selected_terms[:5]:
                print(f"  - {term}")
    
    return all_terms

 # 读取 JSON 文件

with open(clean_data_path, "r", encoding="utf-8") as f:
    data = json.load(f)
# data = data[:10000]

# 如果是列表结构，则提取所有 output 字段
if isinstance(data, list):
    outputs = [item.get("output", "") for item in data]
elif isinstance(data, dict):  # 单条对象
    outputs = [data.get("output", "")]
else:
    raise ValueError("未知的数据格式")

# 提取术语
terms = extract_domain_terms(outputs)

with open(ko_domain_path, "w", encoding="utf-8") as f:
    for term in terms:
        f.write(term + "\n")

print(f"\n 成功提取 {len(terms)} 个术语，并保存到 {ko_domain_path}")

开始提取领域术语...

文本语言分布:
- ko: 21245 条

处理ko语言文本...


并行提取ko语言术语: 100%|██████████| 21245/21245 [41:40<00:00,  8.50it/s]  


ko语言提取了 2000 个术语
ko语言术语示例:
  - 일
  - 등
  - 기자
  - 것
  - 경제

 成功提取 2000 个术语，并保存到 /root/ko_text/ko_domain.txt


## 数据集获取：数据处理成适合MLM的格式  

#### 功能描述  

主要用于加载和预处理用于大语言模型微调的数据。它将原始JSON格式的训练数据转换为适合模型训练的格式,并创建训练和验证数据加载器。  


#### 处理流程  

1. **数据加载与格式化**  
   - 从JSON文件加载原始数据  
   - 将每个样本格式化为"### Instruction:\n{prompt}\n\n### Response:\n{output}"的形式  
   - 记录每个样本中"### Response:\n"的字符位置  

2. **分词处理**  
   - 使用tokenizer对文本进行分词  
   - 设置最大长度并进行截断  
   - 使用max_length进行填充  
   - 获取字符到token的映射信息  

3. **标签准备**  
   - 通过字符位置找到response起始的token索引  
   - 创建标签序列:  
     - response之前的token标记为-100(忽略)  
     - response部分保留原token_id  

4. **数据集创建与划分**  
   - 使用Huggingface的Dataset类创建数据集  
   - 按照设定比例划分训练集和验证集  
   - 将数据格式转换为PyTorch张量  

5. **DataLoader创建**  
   - 实现collate_fn确保batch中序列长度一致  
   - 创建训练和验证数据加载器  
   - 设置batch_size为16 

#### 返回值  

返回一个元组,包含:  
- train_dataloader: 训练数据加载器  
- val_dataloader: 验证数据加载器

In [10]:
import json
from datasets import Dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

def load_and_prepare_data(json_path, tokenizer, max_length=512, val_ratio=0.05):
    """加载并预处理微调数据（标准LLM微调流程）"""
    with open(json_path, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)
    
    # 记录每个样本中"### Response:\n"的字符位置
    formatted_texts = []
    response_char_positions = []
    for item in raw_data:
        prompt = item['instruction']
        output = item['output']
        formatted_text = f"### Instruction:\n{prompt}\n\n### Response:\n{output}"
        formatted_texts.append(formatted_text)
        # 找到"### Response:\n"的字符位置
        response_start_char = formatted_text.find("### Response:\n") + len("### Response:\n")
        response_char_positions.append(response_start_char)
    
    # 分词处理
    tokenized_data = tokenizer(
        formatted_texts,
        max_length=max_length,
        truncation=True,
        padding="max_length",  # 修改：使用max_length填充
        add_special_tokens=True,
        return_offsets_mapping=True  # 需要字符到token的映射
    )
    
    # 准备标签
    labels = []
    for i in range(len(tokenized_data["input_ids"])):
        input_ids = tokenized_data["input_ids"][i]
        offsets = tokenized_data["offset_mapping"][i]
        
        # 通过字符位置找到response起始的token索引
        response_start_token = None
        for token_idx, (char_start, char_end) in enumerate(offsets):
            if char_start >= response_char_positions[i]:
                response_start_token = token_idx
                break
        
        # 如果未找到（例如被截断），则设为整个序列
        if response_start_token is None:
            response_start_token = len(input_ids)
        
        # 创建label：只保留response部分的token_id
        label = [-100] * len(input_ids)
        label[response_start_token:] = input_ids[response_start_token:]
        labels.append(label)
    
    # 移除offset_mapping（不再需要）
    tokenized_data.pop("offset_mapping")
    
    # 创建数据集
    dataset = Dataset.from_dict({
        "input_ids": tokenized_data["input_ids"],
        "attention_mask": tokenized_data["attention_mask"],
        "labels": labels
    })
    
    # 划分训练验证集
    split_dataset = dataset.train_test_split(test_size=val_ratio, seed=42)
    
    # 转换为PyTorch张量格式
    def set_format(ds):
        ds.set_format(type='torch', columns=["input_ids", "attention_mask", "labels"])
        return ds
    
    train_dataset = set_format(split_dataset["train"])
    val_dataset = set_format(split_dataset["test"])
    
    # 修正后的collate_fn - 确保所有序列长度一致
    def collate_fn(batch):
        # 将batch中的每个tensor转为列表以便处理
        input_ids = [item["input_ids"].tolist() for item in batch]
        attention_mask = [item["attention_mask"].tolist() for item in batch]
        labels = [item["labels"].tolist() for item in batch]
        
        # 找出最大长度
        max_len = max(len(ids) for ids in input_ids)
        
        # 进行右侧填充
        for i in range(len(batch)):
            pad_len = max_len - len(input_ids[i])
            if pad_len > 0:
                input_ids[i] += [tokenizer.pad_token_id] * pad_len
                attention_mask[i] += [0] * pad_len
                labels[i] += [-100] * pad_len  # 使用-100填充标签
        
        # 转换回tensor
        result = {
            "input_ids": torch.tensor(input_ids),
            "attention_mask": torch.tensor(attention_mask),
            "labels": torch.tensor(labels)
        }
        
        return result
    
    # 创建DataLoader
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=16,
        shuffle=True,
        collate_fn=collate_fn
    )
    val_dataloader = DataLoader(
        val_dataset,
        batch_size=16,
        collate_fn=collate_fn
    )
    
    return train_dataloader, val_dataloader



## 模型加载与训练——qwen2.5-1.5B模型微调 
  1. 模型准备  
     - 加载qwen2.5-1.5B基础模型  
     - 配置tokenizer  
     - 设置LoRA参数  
  2. 训练配置  
     - 学习率设置  
     - 批次大小选择  
     - 训练轮次确定  
     - 优化器选择  
  3. 训练过程  
     - 梯度更新  
     - 学习率调度  
     - 模型保存  
     - 训练监控

In [11]:
# 设置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'training_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# 设置随机种子以确保结果可重现
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)



def find_optimal_lora_config(model, train_dataloader, val_dataloader, device, evaluator, batch_size):
    """搜索最优的LoRA配置，使用小数据集快速搜索"""
    configs = [
        {"r": 64, "alpha": 128},
        {"r": 32, "alpha": 64},
        {"r": 16, "alpha": 32},
        {"r": 8, "alpha": 16},
        {"r": 4, "alpha": 8}
    ]
    
    best_perplexity = float('inf')
    best_config = None
    
    # 从训练集和验证集中各取100条数据创建小数据集
    small_train_data = []
    small_val_data = []
    
    # 收集小训练集
    train_iter = iter(train_dataloader)
    for _ in range(min(256//batch_size, len(train_dataloader))):  # 25个batch * 4 = 100条数据
        try:
            batch = next(train_iter)
            small_train_data.append(batch)
        except StopIteration:
            break
    
    # 收集小验证集
    val_iter = iter(val_dataloader)
    for _ in range(min(128//batch_size, len(val_dataloader))):
        try:
            batch = next(val_iter)
            small_val_data.append(batch)
        except StopIteration:
            break
    
    logger.info(f"创建了小数据集用于配置搜索：训练集 {len(small_train_data)} 批次，验证集 {len(small_val_data)} 批次")
    
    for config in configs:
        logger.info(f"\n测试LoRA配置: r={config['r']}, alpha={config['alpha']}")
        
        # 配置LoRA
        lora_config = LoraConfig(
            r=config['r'],
            lora_alpha=config['alpha'],
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
            lora_dropout=0.05,
            bias="none",
            task_type=TaskType.CAUSAL_LM
        )
        
        # 创建PEFT模型
        peft_model = get_peft_model(model, lora_config)
        
        # 快速训练和评估
        optimizer = AdamW(peft_model.parameters(), lr=2e-4)
        peft_model.train()
        
        # 在小训练集上训练
        for _ in range(2):  # 只训练2个epoch
            for batch in small_train_data:
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = peft_model(**batch)
                loss = outputs.loss
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
        
        # 在小验证集上评估
        peft_model.eval()
        total_loss = 0
        total_tokens = 0
        with torch.no_grad():
            for batch in small_val_data:
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = peft_model(**batch)
                total_loss += outputs.loss.item() * batch["input_ids"].size(0)
                total_tokens += batch["input_ids"].ne(evaluator.tokenizer.pad_token_id).sum().item()
        # 计算困惑度，原理是计算验证集的损失除以验证集的总token数量，然后取指数，目的是衡量模型的困惑度
        perplexity = torch.exp(torch.tensor(total_loss / total_tokens))
        logger.info(f"配置性能 - 困惑度: {perplexity:.2f}")
        
        if perplexity < best_perplexity:
            best_perplexity = perplexity
            best_config = config
            logger.info(f"找到新的最佳配置！")
    
    return best_config

def convert_metrics_to_json_serializable(metrics):
    """将指标转换为JSON可序列化的格式"""
    if isinstance(metrics, dict):
        return {k: convert_metrics_to_json_serializable(v) for k, v in metrics.items()}
    elif isinstance(metrics, list):
        return [convert_metrics_to_json_serializable(v) for v in metrics]
    elif isinstance(metrics, (torch.Tensor, np.ndarray)):
        return metrics.item() if metrics.size == 1 else metrics.tolist()
    elif isinstance(metrics, (int, float, str, bool)):
        return metrics
    elif metrics is None:
        return None
    else:
        return str(metrics)

def main():
    # 设置随机种子
    set_seed(42)
    
    # 检查CUDA
    assert torch.cuda.is_available(), "需要CUDA支持"
    device = torch.device("cuda:0")
    
    print(os.environ.get('HF_ENDPOINT'))
    print(os.environ.get('HF_ENDPOINT'))
    print(os.environ.get('HF_ENDPOINT'))

    # 加载模型和分词器
    model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        trust_remote_code=True,
        torch_dtype=torch.float16,
    
        
    ).to(device)
    
    tokenizer = AutoTokenizer.from_pretrained(
        base_model_path,
        trust_remote_code=True,
    
    )
    tokenizer.pad_token = tokenizer.eos_token
    
    train_dataloader, val_dataloader = load_and_prepare_data(clean_data_path, tokenizer)
    

    # 初始化评估器
    evaluator = DomainEvaluator(tokenizer, device)
    
    # # 寻找最优LoRA配置
    # logger.info("开始寻找最优LoRA配置...")
    # best_config = find_optimal_lora_config(model, train_dataloader, val_dataloader, device, evaluator, batch_size)
    # logger.info(f"找到最优LoRA配置: r={best_config['r']}, alpha={best_config['alpha']}")
    
    # # 使用最优配置创建LoRA模型
    # lora_config = LoraConfig(
    #     r=best_config['r'],
    #     lora_alpha=best_config['alpha'],
    #     target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    #     lora_dropout=0.1,
    #     bias="none",
    #     task_type=TaskType.CAUSAL_LM
    # )
    
    lora_config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_dropout=lora_dropout,
        bias="none",
        task_type=TaskType.CAUSAL_LM
    )

    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()
    
    # 训练配置
    optimizer = AdamW(model.parameters(), lr=lr)
    num_training_steps = len(train_dataloader) * num_epochs
    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps
    )
    
    # 从domain_terms_arabic.txt构建评估提示
    with open(ko_domain_path, "r", encoding="utf-8") as f:
        domain_terms = [line.strip() for line in f if line.strip()]
    
    # # 构建基于领域术语的评估提示
    # unlabeled_eval_prompts = {
    #     "ko": [
    #         f"请解释韩语中'{term}'这个术语的含义和用法。" for term in random.sample(domain_terms, 5)
    #     ] + [
    #         f"请用韩语语写一段话，包含以下术语：{', '.join(random.sample(domain_terms, 3))}",
    #         f"在技术领域中，'{random.choice(domain_terms)}'和'{random.choice(domain_terms)}'这两个术语有什么联系？",
    #         f"请用韩语描述'{random.choice(domain_terms)}'在现代技术发展中的应用。",
    #         f"请生成一段带有韩语专业术语的文本",
    #         f"请生成一段带有韩语专业术语的文本",
    #         f"请生成一段带有韩语专业术语的文本",
    #         f"请生成一段带有韩语专业术语的文本",
    #         f"请生成一段带有韩语专业术语的文本",
    #         f"请生成一段带有韩语专业术语的文本",
    #         f"请生成一段带有韩语专业术语的文本",
    #         f"请生成一段带有韩语专业术语的文本"
    #     ]
    # }
    
    # 训练循环
    best_metrics = {
        "val_perplexity": float('inf'),
        "domain_adaptation": 0
    }
    metrics_log = []
    
    first = 0
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
        print(len(train_dataloader))
        for step, batch in enumerate(progress_bar):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            total_loss += loss.item()
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            
            progress_bar.set_postfix({"loss": loss.item()})
            
            # 评估
            if step % eval_steps == 0 and step > 0:
                model.eval()
                
                # 1. 计算验证集困惑度
                val_perplexity = evaluator.calculate_domain_perplexity(model, val_dataloader, 0.3)
                
                # 2. 评估领域适应性
                domain_metrics = {}
                domain_metrics["ko"] = evaluator.evaluate_domain_adaptation(model, val_dataloader, sample_ratio=1.0)
                print("metrics:", domain_metrics)
                
                # 3. 计算综合指标
                avg_domain_score = np.mean([
                    m["avg_term_coverage"] * 0.2 +
                    m["avg_term_density"] * 0.2 +
                    m["avg_response_quality"] * 0.2 + 
                    m["avg_bleu"] * 0.1 + 
                    m["avg_rouge-1"] * 0.1 + 
                    m["avg_rouge-2"] * 0.1 + 
                    m["avg_rouge-l"] * 0.1 
                    for m in domain_metrics.values()
                ])
                
                # 记录当前学习率
                current_lr = optimizer.param_groups[0]["lr"]
                
                metrics = {
                    "epoch": epoch + 1,
                    "step": step,
                    "val_perplexity": val_perplexity.item() if isinstance(val_perplexity, torch.Tensor) else float(val_perplexity),
                    "domain_adaptation_score": float(avg_domain_score),
                    "learning_rate": float(current_lr),
                    "domain_metrics": convert_metrics_to_json_serializable(domain_metrics)
                }
                metrics_log.append(metrics)
                
                logger.info(f"\n验证集困惑度: {val_perplexity:.4f}")
                logger.info(f"领域适应性得分: {avg_domain_score:.4f}")
                logger.info(f"当前学习率: {current_lr:.6f}")
                
                # 4. 保存最佳模型
                combined_score = avg_domain_score/val_perplexity
                if combined_score > best_metrics["domain_adaptation"]/best_metrics["val_perplexity"]:
                    best_metrics["val_perplexity"] = float(val_perplexity)
                    best_metrics["domain_adaptation"] = float(avg_domain_score)
                    model.save_pretrained(best_model_path)
                    logger.info(f"保存新的最佳模型！困惑度={val_perplexity:.4f}, 领域得分={avg_domain_score:.4f}")
                
                model.train()

        # 每个epoch结束保存检查点
        checkpoint_path = f"/root/ko_text/saves/qwen-lora-checkpoint-{epoch+1}"
        try:
            model.save_pretrained(checkpoint_path)
            logger.info(f"已保存Epoch {epoch+1}检查点")
        except Exception as e:
            logger.error(f"保存检查点时出错: {e}")
        
        avg_loss = total_loss / len(train_dataloader)
        logger.info(f"Epoch {epoch+1} 平均损失: {avg_loss:.4f}")
    
    # 保存最终模型和训练指标
    try:
        model.save_pretrained(final_model_path)
        with open('training_metrics.json', 'w', encoding='utf-8') as f:
            json.dump(metrics_log, f, ensure_ascii=False, indent=2)
        logger.info("训练完成！已保存最终模型和训练指标")
    except Exception as e:
        logger.error(f"保存最终结果时出错: {e}")

# 19. 主程序入口
if __name__ == "__main__":
    # 打印环境信息
    logger.info(f"PyTorch version: {torch.__version__}")
    logger.info(f"CUDA available: {torch.cuda.is_available()}")
    logger.info(f"CUDA version: {torch.version.cuda}")
    logger.info(f"GPU count: {torch.cuda.device_count()}")
    logger.info(f"Current GPU: {torch.cuda.current_device()}")
    logger.info(f"GPU name: {torch.cuda.get_device_name(0)}")
    
    lora_r = 32
    lora_alpha = 64
    lora_dropout = 0.05
    num_epochs = 3
    num_warmup_steps = 20
    eval_steps = 1000  # 每1000步评估一次
    batch_size = 16
    lr = 5e-4
    best_model_path = "/root/ko_text/saves/best"
    final_model_path = "/root/ko_text/saves/final"
    base_model_path = "/root/.cache/modelscope/hub/models/Qwen/Qwen2.5-1.5B-Instruct"

    main()

2025-08-08 13:53:14,728 - INFO - PyTorch version: 2.8.0+cu128
2025-08-08 13:53:14,958 - INFO - CUDA available: True
2025-08-08 13:53:14,959 - INFO - CUDA version: 12.8
2025-08-08 13:53:15,026 - INFO - GPU count: 1
2025-08-08 13:53:15,036 - INFO - Current GPU: 0
2025-08-08 13:53:15,036 - INFO - GPU name: NVIDIA A100-SXM4-80GB


None
None
None


2025-08-08 13:53:30,394 - INFO - Use pytorch device_name: cuda:0
2025-08-08 13:53:30,395 - INFO - Load pretrained SentenceTransformer: /root/.cache/modelscope/hub/models/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2


trainable params: 8,716,288 || all params: 1,552,430,592 || trainable%: 0.5615


Epoch 1:   0%|          | 0/1262 [00:00<?, ?it/s]

...

Epoch 2: 100%|██████████| 1262/1262 [23:55<00:00,  1.14s/it, loss=1.94]    
2025-08-08 14:40:58,789 - INFO - 已保存Epoch 2检查点
2025-08-08 14:40:58,793 - INFO - Epoch 2 平均损失: 1.6966
Epoch 3:   0%|          | 0/1262 [00:00<?, ?it/s]

1262


Batches: 100%|██████████| 1/1 [00:00<00:00, 96.14it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 136.60it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 120.38it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 141.92it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 101.78it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 135.85it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 114.23it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 141.51it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 110.07it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 142.33it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 111.02it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 140.07it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 115.15it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 142.17it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 108.27it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 135.08it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 111.88it/s]

Batches: 100%|██████████| 1/1 [0

metrics: {'ko': {'avg_term_coverage': np.float64(0.07296519285042333), 'avg_term_density': np.float64(0.2165156330388828), 'avg_response_quality': np.float64(0.7997989903220789), 'avg_bleu': np.float64(0.841969061137037), 'avg_rouge-1': np.float64(0.9123066499798942), 'avg_rouge-2': np.float64(0.907620809761489), 'avg_rouge-l': np.float64(0.9075666556877058)}}


2025-08-08 15:00:07,027 - INFO - 保存新的最佳模型！困惑度=1.0070, 领域得分=0.5748
Epoch 3: 100%|██████████| 1262/1262 [21:26<00:00,  1.02s/it, loss=2.96]    
2025-08-08 15:02:25,722 - INFO - 已保存Epoch 3检查点
2025-08-08 15:02:25,727 - INFO - Epoch 3 平均损失: 2.4020
2025-08-08 15:02:25,953 - INFO - 训练完成！已保存最终模型和训练指标


## 模型的更新（权重合并）与使用，这里使用的是best权重

这里加载"/root/ko_text/saves/best"目录下的最佳模型  
观察该模型权重性能  

#### 先执行专业词解释，后执行文本生成任务

In [12]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType
import json
import logging
from datetime import datetime
import numpy as np
from pathlib import Path

merged_model_path = "/root/ko_text/saves/meraged"

# 设置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'merge_and_save_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

def convert_metrics_to_json_serializable(metrics):
    """将指标转换为JSON可序列化的格式"""
    if isinstance(metrics, dict):
        return {k: convert_metrics_to_json_serializable(v) for k, v in metrics.items()}
    elif isinstance(metrics, list):
        return [convert_metrics_to_json_serializable(v) for v in metrics]
    elif isinstance(metrics, (torch.Tensor, np.ndarray)):
        return metrics.item() if metrics.size == 1 else metrics.tolist()
    elif isinstance(metrics, (int, float, str, bool)):
        return metrics
    elif metrics is None:
        return None
    else:
        return str(metrics)

def save_metrics_and_merge_model():
    """保存训练指标并合并模型"""
    try:
        # 1. 保存训练指标（如果有）
        if 'metrics_log' in globals():
            logger.info("正在保存训练指标...")
            metrics_log_serializable = [convert_metrics_to_json_serializable(m) for m in metrics_log]
            with open('training_metrics.json', 'w', encoding='utf-8') as f:
                json.dump(metrics_log_serializable, f, ensure_ascii=False, indent=2)
            logger.info("训练指标已保存到 training_metrics.json")
        
        # 2. 加载和合并模型
        logger.info("开始加载模型...")
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # 加载基础模型
        logger.info(f"加载基础模型: {base_model_path}")
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_path,
            trust_remote_code=True,
            torch_dtype=torch.float16
        ).to(device)
        
        # 加载tokenizer
        tokenizer = AutoTokenizer.from_pretrained(
            base_model_path,
            trust_remote_code=True
        )
        
        # 配置LoRA
        lora_config = LoraConfig(
            r=lora_r,
            lora_alpha=lora_alpha,
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
            lora_dropout=lora_dropout,
            bias="none",
            task_type=TaskType.CAUSAL_LM
        )
        
        # 创建PEFT模型
        logger.info("应用LoRA配置...")
        model = get_peft_model(base_model, lora_config)
        
        # 加载最佳检查点
        logger.info(f"加载最佳检查点: {best_model_path}")
        model.load_adapter(best_model_path, adapter_name="arabic_adapter")
        
        # 合并权重
        logger.info("合并模型权重...")
        merged_model = model.merge_and_unload()
        
        # 保存合并后的模型
        logger.info(f"保存合并后的模型到: {merged_model_path}")
        merged_model.save_pretrained(merged_model_path)
        
        # 保存tokenizer
        logger.info("保存tokenizer...")
        tokenizer.save_pretrained(merged_model_path)
        
        logger.info("所有操作完成！")
        
        # 返回模型和tokenizer以供后续使用
        return merged_model, tokenizer
        
    except Exception as e:
        logger.error(f"处理过程中出错: {e}")
        raise

def test_merged_model(model, tokenizer, test_prompts=None):
    """测试合并后的模型"""
    if test_prompts is None:
        # 使用domain_terms.txt中的术语构建测试提示
        try:
            with open(ko_domain_path, "r", encoding="utf-8") as f:
                domain_terms = [line.strip() for line in f if line.strip()]
            
            # 随机选择一些术语构建测试提示
            import random
            selected_terms = random.sample(domain_terms, min(10, len(domain_terms)))
            test_prompts = [
                f"请解释韩语中'{term}'这个术语的含义。" for term in selected_terms
            ]
        except Exception as e:
            logger.warning(f"无法加载domain_terms.txt，使用默认测试提示: {e}")
            test_prompts = [f"请生成一段带有韩语专业术语的文本"] * 10
    
    logger.info("\n开始测试合并后的模型...")
    device = next(model.parameters()).device
    
    for prompt in test_prompts:
        logger.info(f"\n测试提示: {prompt}")
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=512,
                num_return_sequences=1,
                temperature=0.7,
                do_sample=True,
                top_p=0.95,
                repetition_penalty=1.1
            )
        
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        logger.info(f"模型回答: {response}\n")
        logger.info("-" * 50)

if __name__ == "__main__":
    # 打印环境信息
    logger.info(f"PyTorch version: {torch.__version__}")
    logger.info(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        logger.info(f"CUDA version: {torch.version.cuda}")
        logger.info(f"GPU count: {torch.cuda.device_count()}")
        logger.info(f"Current GPU: {torch.cuda.current_device()}")
        logger.info(f"GPU name: {torch.cuda.get_device_name(0)}")
    
    # 执行合并和保存
    merged_model, tokenizer = save_metrics_and_merge_model()
    
    # 测试模型
    test_merged_model(merged_model, tokenizer) 

2025-08-08 15:05:28,440 - INFO - PyTorch version: 2.8.0+cu128
2025-08-08 15:05:28,443 - INFO - CUDA available: True
2025-08-08 15:05:28,444 - INFO - CUDA version: 12.8
2025-08-08 15:05:28,449 - INFO - GPU count: 1
2025-08-08 15:05:28,453 - INFO - Current GPU: 0
2025-08-08 15:05:28,457 - INFO - GPU name: NVIDIA A100-SXM4-80GB
2025-08-08 15:05:28,463 - INFO - 开始加载模型...
2025-08-08 15:05:28,470 - INFO - 加载基础模型: /root/.cache/modelscope/hub/models/Qwen/Qwen2.5-1.5B-Instruct
2025-08-08 15:05:31,480 - INFO - 应用LoRA配置...
2025-08-08 15:05:32,198 - INFO - 加载最佳检查点: /root/ko_text/saves/best
2025-08-08 15:05:33,387 - INFO - 合并模型权重...
2025-08-08 15:05:33,433 - INFO - 保存合并后的模型到: /root/ko_text/saves/meraged
2025-08-08 15:05:47,371 - INFO - 保存tokenizer...
2025-08-08 15:05:47,675 - INFO - 所有操作完成！
2025-08-08 15:05:47,686 - INFO - 
开始测试合并后的模型...
2025-08-08 15:05:47,687 - INFO - 
测试提示: 请解释韩语中'정착'这个术语的含义。
2025-08-08 15:05:52,309 - INFO - 模型回答: 请解释韩语中'정착'这个术语的含义。 '정착'是韩语中的一个术语，通常用于描述一个人在某个地方稳定下来的过程或状态。这个词可以从以

## 对比不同权重（基座模型）的模型性能效果  

In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 设置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'base_model_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 加载合并后的模型和tokenizer
model = AutoModelForCausalLM.from_pretrained(
            base_model_path,
            trust_remote_code=True,
            torch_dtype=torch.float16
        ).to(device)
        
# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    base_model_path,
    trust_remote_code=True
)
logger.info(f"正在加载基座模型: {base_model_path}")

test_prompt = []
# 使用domain_terms.txt中的术语构建测试提示
try:
    with open(ko_domain_path, "r", encoding="utf-8") as f:
        domain_terms = [line.strip() for line in f if line.strip()]
    
    # 随机选择一些术语构建测试提示
    import random
    selected_terms = random.sample(domain_terms, min(10, len(domain_terms)))
    test_prompts = [
        f"请解释韩语中'{term}'这个术语的含义。" for term in selected_terms
    ]
except Exception as e:
    logger.warning(f"无法加载domain_terms.txt，使用默认测试提示: {e}")
    test_prompts = [f"请生成一段带有韩语专业术语的文本"] * 10

logger.info("\n开始测试基础模型...")
device = next(model.parameters()).device

for prompt in test_prompts:
    logger.info(f"\n测试提示: {prompt}")
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=512,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            top_p=0.95,
            repetition_penalty=1.1
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    logger.info(f"模型回答: {response}\n")
    logger.info("-" * 50)



2025-08-08 15:59:14,228 - INFO - 正在加载基座模型: /root/.cache/modelscope/hub/models/Qwen/Qwen2.5-1.5B-Instruct
2025-08-08 15:59:14,241 - INFO - 
开始测试基础模型...
2025-08-08 15:59:14,243 - INFO - 
测试提示: 请解释韩语中'현실'这个术语的含义。
2025-08-08 15:59:19,799 - INFO - 模型回答: 请解释韩语中'현실'这个术语的含义。 '현실' (현실) in Korean is the term used to describe something that actually exists or is real, as opposed to an illusion, dream, or fictional creation. It refers to objective reality and includes tangible objects, events, laws of nature, etc. The word literally means "reality" in English.

In more specific contexts:
1. **현실** (현실) - Real life
2. **현실** (현실) - Current state or situation
3. **현실** (현실) - Actual experience
4. **현실** (현실) - Objective fact

For example: 
- 현실이란 실제로 존재하는 것을 말한다.
  (현실은 실존한다는 뜻이다.)
  ("Reality means what really exists.")
- 그는 현실에서 흘러나오는 물음표에 답을 못 찾았다.
  (그는 현실에서 나온 질문에 답을 찾지 못했다.)
  ("He couldn't find an answer to the question that flowed from reality.")

The concept of '현실' (현실) encapsulates the id

## 总结  
本项目主要包括
1. 数据集准备与清洗  
2. 编码  
3. 确定评估指标与训练方法  
4. 模型下载（部署）与训练  
5. 模型评估  

### 数据集准备与清洗  
我们通过opendatalab提供的原始数据，完成了“专业名词提取+IO数据对”的构造，IO数据对的格式满足"### Instruction:\n{prompt}\n\n### Response:\n{output}"  
正确的数据格式才能带来有效的微调效果  


### 编码  
qwen作为经典的因果语言，需要将数据集编码成符合因果语言训练的格式，该环节最终输出单向编码的数据，单向编码可以让模型在训练过程中学习训练集中的上下文语义关系  

### 确定评估指标与训练方法  
我们通过困惑度（perplexity）与 领域适应性评估（术语覆盖率、术语密度、响应质量）和bleu与rouge作为评估指标，这些指标也是LLM中常用的评估指标  

### 模型下载（部署）与训练  
使用modelscope而不是hf下载，加快了模型下载速度，并且通过Lora微调方法在加快训练速度同时，保证训练质量  

### 模型评估  
我们对比了基座与微调后模型性能，best权重目录下的模型性能表现更好，也证明了评估指标与微调方法的有效性